In [0]:
df = spark.range(0,5)
df.write.format("delta").save("/FirstDeltaTable")


In [0]:
%sql
DESCRIBE DETAIL "/FirstDeltaTable"

In [0]:
%sql
select * from delta.`/FirstDeltaTable`;


In [0]:
data = spark.range(0,5)
data.write.format("delta").mode("overwrite").save("/SecondDeltaTable")
more_data = spark.range(5,11)
more_data.write.format("delta").mode("append").save("/SecondDeltaTable")

In [0]:
%sql
select * from SecondDeltaTable

In [0]:
# optimisation options in spark for big datasets

spark.sql("OPTIMIZE '/SecondDeltaTable' ") #compress data for optimisation
spark.sql("VACUUM '/SecondDeltaTable' RETAIN 170 HOURS ") # delete data older than 170 hours


In [0]:
%sql
DESCRIBE DETAIL "/SecondDeltaTable"

-- it does show its a delta table so NO IDEA!!!!!


In [0]:
%fs ls "dbfs:/" 





Let's try creating delta table using SQL



In [0]:

%sql
create table DeltaTableWithDefinedSchema (
  Country string,
  Continent string,
  Population string
)

In [0]:
%sql
describe detail DeltaTableWithDefinedSchema
-- location: dbfs:/user/hive/warehouse/deltatablewithdefinedschema


In [0]:
%sql
insert into DeltaTableWithDefinedSchema (Country,Continent) VALUES ('England','Europe');
select * from DeltaTableWithDefinedSchema;


In [0]:
%sql
--insert into DeltaTableWithDefinedSchema (Country,Continent) VALUES ('Greece','Europe');
select * from deltatablewithdefinedschema
-- it did not fail. It inserted record with null for the missing column


In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

In [0]:
spark.conf.set("spark.databricks.delta.schema.enforcement", "true")


In [0]:
%sql
insert into DeltaTableWithDefinedSchema 
SELECT 'Spain','Europe',1,2,3;

/*
any new columns coming in from source get added to the terget table dynamically. 
It can be useful when source has new columns suddenly, but 
drawback is that any subsequent source data that does not have new columns will fail
*/


In [0]:
%sql
update DeltaTableWithDefinedSchema set Country = 'Northern Ireland' where COuntry = 'Wales'

In [0]:
%sql
--describe detail DeltaTableWithDefinedSchema;
describe history DeltaTableWithDefinedSchema;


In [0]:
%sql
select * from DeltaTableWithDefinedSchema


In [0]:
%sql
select * from DeltaTableWithDefinedSchema version as of 16

In [0]:
%sql
select * from DeltaTableWithDefinedSchema timestamp as of '2024-10-23T11:39:58.000+00:00'


In [0]:
%sql
restore table DeltaTableWithDefinedSchema to timestamp as of '2024-10-23T11:39:58.000+00:00'



In [0]:
%sql
select * from DeltaTableWithDefinedSchema